#### CELL-1 코드리뷰 

- <strong>NUM_CLASSES = </strong> -> 설정했던 클래스의 갯수를 입력합니다
- 모델에 첫번째 input layer를 생성할때 필요합니다

In [ ]:
NUM_CLASSES =  ###클래스 갯수를 입력하세요

<br></br>

#### CELL-2 코드리뷰 

- <strong>EPOCH = 150</strong> -> 전체 훈련 데이터를 150번 반복합니다
- <strong>pre_epoch = 0</strong> -> 처음부터 시작합니다
- <strong>BATCH_SIZE = 16</strong> -> 16개의 샘플이 동시에 모델에 입력되고, 이 데이터를 기반으로 모델의 가중치가 업데이트됩니다
- <strong>LR = 0.01</strong> -> 모델이 각 배치마다 가중치를 업데이트할 때 0.01만큼 조정합니다

In [1]:
# Set hyperparameters
EPOCH = 150
pre_epoch = 0
BATCH_SIZE = 16
LR = 0.01

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
import os
import cv2
import numpy as np
print(os.getcwd())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.data = self._load_data()

    def _load_data(self):
        data = []
        for cls in self.classes:
            class_path = os.path.join(self.root_dir, cls)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                data.append((img_path, self.class_to_idx[cls]))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# Import the ResNet-18 model
resnet18 = models.resnet18(pretrained=False)  # Set pretrained=True to load weights pre-trained on ImageNet



In [4]:
# Prepare dataset and preprocessing
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # Change input size
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),  # Change input size
    transforms.ToTensor(),
])

# Set the directory
current_directory = os.getcwd()
file_path_train = "data"
file_path_test = "data"

custom_dataset_train = CustomDataset(root_dir=file_path_train, transform=transform_train)
custom_dataset_test = CustomDataset(root_dir=file_path_test, transform=transform_test)

trainloader = torch.utils.data.DataLoader(custom_dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(custom_dataset_test, batch_size=8, shuffle=True, num_workers=2)


In [6]:
# Define ResNet18
net = models.resnet18(pretrained=False)  # Import ResNet18 from torchvision.models
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, NUM_CLASSES)
net = net.to(device)

In [7]:
# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

In [ ]:

for epoch in range(pre_epoch, EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainloader, 0):
        # Prepare dataset
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward & backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print accuracy & loss in each batch
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
              % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
    

    model_scripted = torch.jit.script(net)
    model_scripted.save(os.path.join('checkpoints', f'resnet_epoch_{epoch + 1}.pt'))
    
    
    # Get accuracy with the test dataset in each epoch
    print('Waiting Test...')
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            net.eval()
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()

        print('Test\'s accuracy is: %.3f%%' % (100. * correct / total))
print('Training has finished, total epoch is %d' % EPOCH)
